In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from deap import algorithms, base, creator, tools
from pymoo.algorithms.moo.nsga2 import NSGA2
from pymoo.operators.crossover.sbx import SBX
from pymoo.operators.mutation.pm import PM
from pymoo.operators.sampling.rnd import FloatRandomSampling
from pymoo.termination import get_termination
from pymoo.optimize import minimize
from pymoo.visualization.scatter import Scatter
import tensorflow as tf

# Loading the pre-trained BPNN model
qP_model = tf.keras.models.load_model('qP_final_model.h5')
mu_model = tf.keras.models.load_model('mu_final_model.h5')
qS_model = tf.keras.models.load_model('qS_final_model.h5')

# Optimization phase configuration
PHASE_CONFIG = {
    'growth': {
        'variables': [
            {'name': 'AN', 'bounds': [0.64, 0.79]},  # g/L
            {'name': 'pH', 'bounds': [7.28, 7.49]}
        ],
        'models': [mu_model, qS_model]
    },
    'synthesis': {
        'variables': [
            {'name': 'OUR', 'bounds': [8.4, 13.2]},  # mmol/L/h
            {'name': 'TS', 'bounds': [15.8, 18.5]}   # g/L
        ],
        'models': [qP_model, qS_model]
    }
}

class MultiObjectiveOptimizer:
    def __init__(self, phase='growth'):
        self.phase = phase
        self.config = PHASE_CONFIG[phase]
        self.n_var = len(self.config['variables'])
        self.xl = [var['bounds'][0] for var in self.config['variables']]
        self.xu = [var['bounds'][1] for var in self.config['variables']]
        
        # Defining a multi-objective optimization problem
        self.problem = self._create_problem()
        self.algorithm = NSGA2(
            pop_size=50,
            sampling=FloatRandomSampling(),
            crossover=SBX(prob=0.9, eta=15),
            mutation=PM(eta=20),
            eliminate_duplicates=True
        )
        self.termination = get_termination("n_gen", 100)

    def _create_problem(self):
        class OptimizationProblem:
            def __init__(self, xl, xu, n_var, models):
                self.n_var = n_var
                self.xl = np.array(xl)
                self.xu = np.array(xu)
                self.n_obj = 3  # μ, qP, qS
                self.models = models

            def _evaluate(self, x, out, *args, **kwargs):
                # Construct the complete input feature vector
                X = np.zeros((x.shape[0], 7))  
                
                # Optimize variable positions according to stage settings
                if self.phase == 'growth':
                    X[:, 0] = x[:, 0]  # AN
                    X[:, 1] = x[:, 1]  # pH
                    # Other features are set to default or fixed values
                    X[:, 2:] = [0.5, 25, 6.5, 0.8, 12]  # example value
                else:
                    X[:, 2] = x[:, 0]  # OUR
                    X[:, 3] = x[:, 1]  # TS
                    # Other features are set to default or fixed values
                    X[:, [0,1,4,5,6]] = [0.7, 7.3, 0.6, 15, 10]  # example value

                # model prediction
                mu = self.models[0].predict(X, verbose=0).flatten()
                q = self.models[1].predict(X, verbose=0).flatten()
                qP = qP_model.predict(X, verbose=0).flatten()
                
                # Construct the objective function (maximize to minimize)
                f1 = -mu   # maximize μ
                f2 = -qP  # maximize qP
                f3 = -q   # maximize qS
                
                out["F"] = np.column_stack([f1, f2, f3])

        return OptimizationProblem(self.xl, self.xu, self.n_var, self.config['models'])

    def optimize(self):
        res = minimize(
            self.problem,
            self.algorithm,
            self.termination,
            seed=42,
            verbose=True
        )
        return res

    @staticmethod
    def visualize_pareto(res, phase):
        fig = plt.figure(figsize=(12, 8))
        
        # 3D Pareto Front
        ax = fig.add_subplot(111, projection='3d')
        F = res.F
        ax.scatter(-F[:,0], -F[:,1], -F[:,2], c='blue', s=30)
        
        # Labeling Key Points
        extremes = {
            'Point1': np.argmin(F[:,0] + F[:,1]),
            'Point5': np.argmin(F[:,2]),
            'Point4': np.argmin(np.linalg.norm(F, axis=1))
        }
        
        for name, idx in extremes.items():
            ax.scatter(-F[idx,0], -F[idx,1], -F[idx,2], c='red', s=100, marker='*')
            ax.text(-F[idx,0], -F[idx,1], -F[idx,2], name, fontsize=12)
        
        ax.set_xlabel('μ (h⁻¹)')
        ax.set_ylabel('qP (mg/gDCW/h)')
        ax.set_zlabel('qS (mg/gDCW/h)')
        ax.set_title(f'{phase.capitalize()} Phase Pareto Front')
        plt.savefig(f'{phase}_pareto_3d.png', dpi=300)
        plt.close()

# Perform growth stage optimization
print("Optimizing the growth phase...")
growth_optimizer = MultiObjectiveOptimizer(phase='growth')
growth_res = growth_optimizer.optimize()
growth_optimizer.visualize_pareto(growth_res, 'growth')

# Perform GM C1a biosynthesis phase optimization
print("\n Optimize GM C1a biosynthesis phase...")
synthesis_optimizer = MultiObjectiveOptimizer(phase='synthesis')
synthesis_res = synthesis_optimizer.optimize()
synthesis_optimizer.visualize_pareto(synthesis_res, 'synthesis')

# Decision analysis (using the growth phase as an example)
def select_optimal_solution(F):
    # Standardized target value
    F_norm = (F - F.min(axis=0)) / (F.max(axis=0) - F.min(axis=0))
    
    # LINMAP
    ideal = F_norm.min(axis=0)
    linmap_dist = np.linalg.norm(F_norm - ideal, axis=1)
    linmap_idx = np.argmin(linmap_dist)
    
    # TOPSIS
    nadir = F_norm.max(axis=0)
    topsis_score = np.linalg.norm(F_norm - ideal, axis=1) / (
        np.linalg.norm(F_norm - ideal, axis=1) + np.linalg.norm(F_norm - nadir, axis=1))
    topsis_idx = np.argmax(topsis_score)
    
    return linmap_idx, topsis_idx

# Applying decision-making methods
F_growth = -growth_res.F  # Convert back to original target value
linmap_idx, topsis_idx = select_optimal_solution(F_growth)

print(f"\nOptimal Solution Selection for Growth Stage:")
print(f"LINMAP pickpoint: μ={F_growth[linmap_idx,0]:.3f}, qP={F_growth[linmap_idx,1]:.3f}, qS={F_growth[linmap_idx,2]:.3f}")
print(f"TOPSIS pickpoint: μ={F_growth[topsis_idx,0]:.3f}, qP={F_growth[topsis_idx,1]:.3f}, qS={F_growth[topsis_idx,2]:.3f}")

# Save optimal control parameters
optimal_params = {
    'growth': growth_res.X[np.argmin(growth_res.F[:,0])],
    'synthesis': synthesis_res.X[np.argmin(synthesis_res.F[:,1])]
}

np.savez('optimal_control_params.npz', **optimal_params)